What is in this notebook:

Four different functions representing four self proposed mixing rules

These functions predict/calculate a mixed solvent's activity coefficient from two single solvents' activity coefficient 

In [1]:
# Needed libraries

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math
from scipy.optimize import curve_fit
from scipy import integrate
from sklearn.metrics import mean_squared_error

## 1. $\gamma_\pm$ average by mass fraction of solvents

In [2]:
def mass(gamma_A,gamma_B,r,massfrac_B):
    """
    inputs: gamma_A = (array or Series) activity coefficient of salt in solvent A
            gamma_B = (array or Series) activity coefficient of salt in solvent B
            r = (int) mass percentage of solvent B in mixed solvents A+B that we are calculating for
            massfrac_B = (int) 100 if we have 2 pure solvents gamma values, not 100 if we only have mixed solvents A+B values
    """
    avg_mass = (1 - r/massfrac_B)*gamma_A + (r/massfrac_B)*gamma_B
    return avg_mass

## 2. $\gamma_\pm$ average by mole fraction of solvents

In [4]:
def mole(gamma_A,gamma_B,r,massfrac_B,M_A,M_B):
    """
    inputs: gamma_A = (array or Series) activity coefficient of salt in solvent A
            gamma_B = (array or Series) activity coefficient of salt in solvent B
            r = (int) mass percentage of solvent B in mixed solvents A+B that we are calculating for
            molefrac_B = (int) 100 if we have 2 pure solvents gamma values, not 100 if we only have mixed solvents A+B values
            M_A = molecular mass of solvent A (g/mol)
            M_B = molecular mass of solvent B (g/mol)
    """
    if massfrac_B == 100: 
        mol_A = (1 - r/100)/(M_A)
        mol_B = (r/100)/(M_B)
        ratio_B = mol_B/mol_A
        total_molratio_B = ratio_B+1
        y = ratio_B/total_molratio_B
        
        avg_mole = (1 - y)*gamma_A + (y)*gamma_B
    else:
        # calculate the mole ratio for the mixed solvents whose data we will use in the average calculation
        mol_A = (1 - massfrac_B/100)/(M_A)
        mol_B = (massfrac_B/100)/(M_B)
        ratio_B = mol_B/mol_A
        total_molratio_B = ratio_B+1
        x = ratio_B/total_molratio_B
        
        mol_A = (1 - r/100)/(M_A)
        mol_B = (r/100)/(M_B)
        ratio_B = mol_B/mol_A
        total_molratio_B = ratio_B+1
        y = ratio_B/total_molratio_B
        
        avg_mole = (1 - (1/x)*y)*gamma_A + (1/x)*(y)*gamma_B
        
    return avg_mole

## 3. $RT\ ln\ activity$ average by mass fraction of solvents

In [7]:
def RTlna(gamma_A,gamma_B,r,massfrac_B,conc):
    """
    inputs: gamma_A = (array or Series) activity coefficient of salt in solvent A
            gamma_B = (array or Series) activity coefficient of salt in solvent B
            r = (int) mass percentage of solvent B in mixed solvents A+B that we are calculating for
            conc = concentration range corresponding to the experimental gamma_A and gamma_B values
            massfrac_B = (int) 100 if we have 2 pure solvents gamma values, not 100 if we only have mixed solvents A+B values
    """
    RT = 8.314*298 # if Temperature at which experimental data was taken was at room temperature
    avg = (1 - r/massfrac_B)*RT*conc*gamma_A + (r/massfrac_B)*RT*conc*gamma_B
    return avg

## 4. Pitzer parameter average by mass fraction of solvents

In [ ]:
def f_gamma(Aphi,conc,a,b):
    f = -Aphi*((np.sqrt(conc)/(1+b*np.sqrt(conc)))+((2/b)*np.log(1+b*np.sqrt(conc))))
    return f

def B_gamma(beta0,beta1,conc,a,b):
    b = 2*beta0 + (2*beta1/a**2*conc)*(1-(1+a*np.sqrt(conc)-a**2*conc/2)*np.exp(-a*np.sqrt(conc)))
    return b

def C_gamma(Cphi):
    c = 1.5*Cphi
    return c

def AC(Aphi,beta0,beta1,Cphi,conc,a,b):
    lnAC = f_gamma(Aphi,conc,a,b)+B_gamma(beta0,beta1,conc,a,b)*conc+C_gamma(Cphi)*(conc)**2
    return np.exp(lnAC)

def pitzeravg(B0_A,B0_B,B1_A,B1_B,Cphi_A,Cphi0_B,r,massfrac_B,conc,Aphi,a,b):
    """
    This function takes the average of pitzer parameters B0, B1, and Cphi by mass fraction, and then calculating
    the activity coefficient of the mixed solvents data for a chosen concentration range "conc"
    inputs: pitzer_A = (array or Series) pitzer parameters of salt in solvent A
            pitzer_B = (array or Series) pitzer parameters of salt in solvent B
            r = (int) mass percentage of solvent B in mixed solvents A+B that we are calculating for
            conc = chosen concentration range 
            massfrac_B = (int) 100 if we have 2 pure solvents gamma values, not 100 if we only have mixed solvents A+B values
    """
    avg_B0 = (1 - r/massfrac_B)*B0_A + (r/massfrac_B)*B0_B
    avg_B1 = (1 - r/massfrac_B)*B1_A + (r/massfrac_B)*B1_B
    avg_Cphi0 = (1 - r/massfrac_B)*Cphi0_A + (r/massfrac_B)*Cphi0_B
    ans = AC(Aphi,avg_B0,avg_B1,avg_Cphi,conc,a,b)
    return ans